In [1]:
import os
import torch
import numpy as np
import time
import random
from sklearn.model_selection import KFold
import argparse
import timeit
from dataset import prepare_data
import psutil
import torch
from models.tagnet_simple import DeepTAGNet
from memory_capacity_utils import gen_lag_data, compute_memory_capacity_vectorized, get_mem_cap_from_model

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('running on GPU')
else:
    device = torch.device("cpu")
    print('running on CPU')

running on GPU


# Load the dataset

In [21]:
def get_args():
    parser = argparse.ArgumentParser(description='Args for graph predition')
    parser.add_argument('-num_folds', type=int, default=3, help='cv number')
    parser.add_argument('--num_timepoints', type=int, default=3,
                        help='Number of timepoints')
    parser.add_argument('-num_epochs', type=int, default=20, help='number of epochs')
    parser.add_argument('--lr', type=float, default=0.001, help="Learning rate")
    parser.add_argument('--memcap_coef', type=float, default=0.001, help="Memory Capacity Loss Coefficient")
    parser.add_argument('-max_lag', type=int, default=35, help='Lag tao for memory capacity signals')
    parser.add_argument('-save_path',type=str,default = '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_tag_oasis_t/',help='Path to the saved results')
    args, _ = parser.parse_known_args()
    return args

In [22]:
args = get_args()

In [23]:
dataset = np.load('datasets/oasis_data.npy')
dataset = torch.from_numpy(dataset).squeeze()
dataset = dataset.type(torch.FloatTensor)
dataset = np.delete(dataset,88,axis=0)

# Set up the network architecture and train-validate

**Train-validate-functions**

In [26]:
def create_directory_if_not_exists(directory):
    """
    Checks if a specified directory exists, and creates it if it doesn't.

    Args:
    - directory (str): Path of the directory to check and potentially create.
    """
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directory '{directory}' was created.")
    else:
        print(f"Directory '{directory}' already exists.")

# Create the results folders
create_directory_if_not_exists(args.save_path+'real_and_predicted_graphs')
create_directory_if_not_exists(args.save_path+'train_losses/mae_losses')
create_directory_if_not_exists(args.save_path+'train_losses/reservoir_losses')
create_directory_if_not_exists(args.save_path+'train_losses/bio_losses')
create_directory_if_not_exists(args.save_path+'train_losses/tp_losses')
create_directory_if_not_exists(args.save_path+'train_losses/total_losses')
create_directory_if_not_exists(args.save_path+'test_mae_losses')
create_directory_if_not_exists(args.save_path+'test_tp_losses')
create_directory_if_not_exists(args.save_path+'test_memcap_losses')
create_directory_if_not_exists(args.save_path+'test_predicted')
create_directory_if_not_exists(args.save_path+'test_original')
create_directory_if_not_exists(args.save_path+'trained_models')

Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_tag_oasis_t/real_and_predicted_graphs' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_tag_oasis_t/train_losses/mae_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_tag_oasis_t/train_losses/reservoir_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_tag_oasis_t/train_losses/bio_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_tag_oasis_t/train_losses/tp_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_tag_oasis_t/train_losses/total_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_tag_oasis_t/test_mae_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_tag_oasis_t/test_tp_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir

In [27]:
manual_seed = 777
np.random.seed(manual_seed)
random.seed(manual_seed)
torch.manual_seed(manual_seed)

# Reservoir signals
X_train_res_np, y_train_res_np = gen_lag_data(1000, 41, args.max_lag)
X_test_res_np, y_test_res_np = gen_lag_data(500, 42, args.max_lag)
X_train_res = torch.from_numpy(X_train_res_np).unsqueeze(1).to(device, dtype=torch.float64)
X_test_res = torch.from_numpy(X_test_res_np).unsqueeze(1).to(device, dtype=torch.float64)
y_train_res = torch.from_numpy(y_train_res_np).to(device, dtype=torch.float64)
y_test_res = torch.from_numpy(y_test_res_np).to(device, dtype=torch.float64)

running on GPU
TRAIN deterministic algorithms


In [28]:
def validation(model, validation_subjects, mem_cap_data, X_train, y_train, X_test, y_test):
    mael = torch.nn.L1Loss().to(device)
    tp = torch.nn.MSELoss().to(device)
    
    val_mae_loss = np.zeros(args.num_timepoints - 1)
    val_tp_loss = np.zeros(args.num_timepoints - 1)
    mem_cap = np.zeros(args.num_timepoints - 1)
    predicted = np.zeros((validation_subjects.shape[0], args.num_timepoints - 1, 35, 35))
    actual = np.zeros((validation_subjects.shape[0], args.num_timepoints - 1, 35, 35))
    
    model.eval()

    with torch.no_grad():
        for n_subject, data in enumerate(validation_subjects):
            input = data[0]
            for t in range(args.num_timepoints - 1):
                pred = model(input)
                val_mae_loss[t] += mael(pred, data[t + 1])
                val_tp_loss[t] += tp(pred.sum(dim=-1), data[t + 1].sum(dim=-1))
                input = pred
                
                pred_mem_cap = get_mem_cap_from_model(model, pred, 
                                                      X_train, y_train, X_test, y_test)
                actual_mem_cap = torch.tensor(mem_cap_data[n_subject, t + 1]).to(device)
                mem_cap[t] += torch.abs(pred_mem_cap - actual_mem_cap)

                predicted[n_subject, t] = pred.cpu().detach().numpy()
                actual[n_subject, t] = data[t + 1].cpu().detach().numpy()
                

    avg_val_mae_loss = val_mae_loss/len(validation_subjects)
    avg_val_tp_loss = val_tp_loss/len(validation_subjects)
    avg_val_mae_mem_cap = mem_cap/len(validation_subjects)

    return avg_val_mae_loss, avg_val_tp_loss, avg_val_mae_mem_cap, predicted, actual

In [30]:
torch.manual_seed(1)
input_weights = (torch.rand((35, 1), dtype=torch.float64) * 2.0 - 1.0).to(device)

indexes = range(args.num_folds)
kfold = KFold(n_splits=args.num_folds, shuffle=True, random_state=manual_seed)
dataset = dataset.to(device)
actual_mem_caps = np.load('datasets/mem_caps_oasis.npy')
f = 0

for train, test in kfold.split(range(dataset.shape[0])):
    print(
            f'------------------------------------Fold [{f + 1}/{args.num_folds}]-----------------------------------------')
    
    train_data = dataset[train]
    test_data = dataset[test]
    train_mem_cap = actual_mem_caps[train]
    test_mem_cap = actual_mem_caps[test]
    
    validation_split = int(0.8 * len(train_data))
    train_subjects = train_data[:validation_split]
    train_mem_cap_subjects = train_mem_cap[:validation_split]
    validation_subjects = train_data[validation_split:]
    validation_mem_cap_subjects = train_mem_cap[:validation_split]

    model = DeepTAGNet(device=device, input_weights=input_weights).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    mael = torch.nn.L1Loss().to(device)
    tp = torch.nn.MSELoss().to(device)
  
    # Start measuring the epochs time
    epochs_start = time.time()
    for epoch in range(args.num_epochs):
        
        print(f'Epoch [{epoch + 1}/{args.num_epochs}]')
        # Set the model in training mode
        model.train()
        
        # this is our loss for all the data
        mae_loss_overall = []
        tp_loss_overall = []
        mae_mem_cap_overall = []
        
        # loop through the data batches
        for data_id, data in enumerate(train_subjects):

            # zero the gradients
            optimizer.zero_grad()
            mae_loss = 0
            tp_loss = 0
            mem_cap_loss = 0
         
            # loop through the time dependent adj matrices in the batches
            for t in range(args.num_timepoints - 1): 
                pred, output_sig = model(data[t], X_train_res, y_train_res, X_test_res)
                real = data[t + 1]
        
                mae_loss += mael(pred, real)

                # Topological Loss
                tp_loss += tp(pred.sum(dim=-1), real.sum(dim=-1))

                # MAE between predicted graph's mem cap and actual graph's mem cap
                predicted_mem_cap = compute_memory_capacity_vectorized(output_sig, y_test_res)
                actual_mem_cap = torch.tensor(train_mem_cap_subjects[data_id, t + 1], requires_grad=True).to(device)
                mem_cap_loss += mael(predicted_mem_cap, actual_mem_cap)

    
            # Calculate the total MAE Loss for the current batch
            mae_loss = mae_loss / (args.num_timepoints - 1)

            # Calculate the total TP Loss for the current batch
            tp_loss = tp_loss / (args.num_timepoints - 1)

            # Calculate the total MAE between Mem Cap Loss for the current batch
            mem_cap_loss = mem_cap_loss / (args.num_timepoints - 1)
            
            # Append to the total MAE Loss
            mae_loss_overall.append(mae_loss.item())
            tp_loss_overall.append(tp_loss.item())
            mae_mem_cap_overall.append(mem_cap_loss.item())
            
            total_loss = mae_loss + args.memcap_coef * mem_cap_loss
            # Update the weights of the neural network
            total_loss.backward()
            optimizer.step()

        mae_loss_overall = np.mean(np.array(mae_loss_overall))
        tp_loss_overall = np.mean(np.array(tp_loss_overall))
        mae_mem_cap_overall = np.mean(np.array(mae_mem_cap_overall))
        print(f"[Train] MAE Loss: {mae_loss_overall}, TP Loss: {tp_loss_overall}, MAE of Mem Caps Loss: {mae_mem_cap_overall}")
    
        avg_val_mae_loss, avg_val_tp_loss, avg_val_mae_mem_cap, _, _ = validation(model, validation_subjects, validation_mem_cap_subjects,
                                                                                  X_train_res, y_train_res, X_test_res, y_test_res)
        print(f"[Validate] MAE Loss Across Timepoints: {avg_val_mae_loss}")
        print(f"[Validate] TP Loss Across Timepoints: {avg_val_tp_loss}")
        print(f"[Validate] MAE of Mem Caps Across Timepoints: {avg_val_mae_mem_cap}")

    
    epochs_end = time.time() - epochs_start
    print()
    print(f'epochs finished with time:{epochs_end}')
    print()
    process = psutil.Process(os.getpid())
    print(f"Current memory usage: {process.memory_info().rss / 1024 ** 2:.2f} MB")
    print()
    
    avg_test_mae_loss, avg_test_tp_loss, avg_test_mem_cap, predicted, original = validation(model, test_data, test_mem_cap,
                                                                                            X_train_res, y_train_res, X_test_res, y_test_res)
    print(f"[Test] MAE Loss Across Timepoints: {avg_test_mae_loss}")
    print(f"[Test] TP Loss Across Timepoints: {avg_test_tp_loss}")
    print(f"[Test] MAE of Mem Caps Across Timepoints: {avg_test_mem_cap}")
    np.save(args.save_path+f"test_mae_losses/mae_test_loss_fold_{f}", avg_test_mae_loss)
    np.save(args.save_path+f"test_tp_losses/tp_test_loss_fold_{f}", avg_test_mae_loss)
    np.save(args.save_path+f"test_memcap_losses/memcap_test_loss_fold_{f}", avg_test_mem_cap)
    np.save(args.save_path+f"test_predicted/predicted_fold_{f}", predicted)
    np.save(args.save_path+f"test_original/original_fold_{f}", original)

    torch.save(model.state_dict(),
               args.save_path +f'trained_models/model_fold_{f}')
    f += 1
    

------------------------------------Fold [1/3]-----------------------------------------
Epoch [1/20]
[Train] MAE Loss: 0.051869647266964115, TP Loss: 2.7909322972098987, MAE of Mem Caps Loss: 0.3469158399047822
[Validate] MAE Loss Across Timepoints: [0.06756175 0.0587244 ]
[Validate] TP Loss Across Timepoints: [9.07310893 1.58566043]
[Validate] MAE of Mem Caps Across Timepoints: [0.59894944 1.05358912]
Epoch [2/20]
[Train] MAE Loss: 0.042006098789473374, TP Loss: 2.1680755148331325, MAE of Mem Caps Loss: 0.26041093938888343
[Validate] MAE Loss Across Timepoints: [0.0597182 0.0491255]
[Validate] TP Loss Across Timepoints: [8.05749613 1.32148768]
[Validate] MAE of Mem Caps Across Timepoints: [0.48321863 0.67330141]
Epoch [3/20]
[Train] MAE Loss: 0.03752154273291429, TP Loss: 1.9044502516587576, MAE of Mem Caps Loss: 0.2132657896545903
[Validate] MAE Loss Across Timepoints: [0.05650959 0.04462663]
[Validate] TP Loss Across Timepoints: [7.67097371 1.17447637]
[Validate] MAE of Mem Caps Acr

[Validate] MAE Loss Across Timepoints: [0.04925236 0.04544659]
[Validate] TP Loss Across Timepoints: [6.51850789 1.47541339]
[Validate] MAE of Mem Caps Across Timepoints: [0.70525191 0.74941318]
Epoch [6/20]
[Train] MAE Loss: 0.030281432314465444, TP Loss: 1.3462646630903086, MAE of Mem Caps Loss: 0.1536133036965618
[Validate] MAE Loss Across Timepoints: [0.04853174 0.04430558]
[Validate] TP Loss Across Timepoints: [6.65085042 1.5210804 ]
[Validate] MAE of Mem Caps Across Timepoints: [0.62381232 0.73376525]
Epoch [7/20]
[Train] MAE Loss: 0.02969870769108335, TP Loss: 1.3502998006840548, MAE of Mem Caps Loss: 0.1620227542858638
[Validate] MAE Loss Across Timepoints: [0.04842243 0.04420375]
[Validate] TP Loss Across Timepoints: [6.67119598 1.55056127]
[Validate] MAE of Mem Caps Across Timepoints: [0.65365684 0.71358729]
Epoch [8/20]
[Train] MAE Loss: 0.028842964923630157, TP Loss: 1.2871869919200738, MAE of Mem Caps Loss: 0.14408286745279705
[Validate] MAE Loss Across Timepoints: [0.0480

[Train] MAE Loss: 0.029347951523959637, TP Loss: 1.5542741266389688, MAE of Mem Caps Loss: 0.15442234599647026
[Validate] MAE Loss Across Timepoints: [0.03925946 0.04315714]
[Validate] TP Loss Across Timepoints: [1.28736341 1.41692686]
[Validate] MAE of Mem Caps Across Timepoints: [0.76180198 0.82601963]
Epoch [11/20]
[Train] MAE Loss: 0.029128542418281236, TP Loss: 1.5393408415218195, MAE of Mem Caps Loss: 0.15521880615232428
[Validate] MAE Loss Across Timepoints: [0.03923756 0.04313397]
[Validate] TP Loss Across Timepoints: [1.31493962 1.43334937]
[Validate] MAE of Mem Caps Across Timepoints: [0.7554889  0.81381496]
Epoch [12/20]
[Train] MAE Loss: 0.029136485404645404, TP Loss: 1.5725595712661744, MAE of Mem Caps Loss: 0.15204986128241776
[Validate] MAE Loss Across Timepoints: [0.03961353 0.04308587]
[Validate] TP Loss Across Timepoints: [1.62611711 1.79081643]
[Validate] MAE of Mem Caps Across Timepoints: [0.63231879 0.79571783]
Epoch [13/20]
[Train] MAE Loss: 0.028642309193188945, 